In [1]:
from os import chdir, path, getcwd
for i in range(10):
    if path.isfile("checkcwd"):
        break
    chdir(path.pardir)
if path.isfile("checkcwd"):
    pass
else:
    raise Exception("Something went wrong. cwd=" + getcwd())

In [2]:
from tqdm.auto import tqdm
from collections import defaultdict
import sys

import requests
import random
from bs4 import BeautifulSoup
from json import loads
from html import unescape
import pandas as pd

import concurrent.futures

In [3]:
RATINGS_DIR = 'resources/ratings'


USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:131.0) Gecko/20100101 Firefox/131.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:130.0) Gecko/20100101 Firefox/130.0',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/109.0',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.170 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_0) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Safari/605.1.15'
]


In [4]:
import os
rating_files = os.listdir(RATINGS_DIR)
len(rating_files)

43

In [5]:
rating_filepaths = [os.path.join(RATINGS_DIR, f) for f in rating_files]
uids = defaultdict(int)
fids = defaultdict(int)
for filepath in tqdm(rating_filepaths):
    with open(filepath, 'r') as fr:
        lines = fr.readlines()
    for line in lines:
        splitted_line = line.split(sep=',')
        uids[splitted_line[0]] += 1
        fids[splitted_line[1]] += 1

  0%|          | 0/43 [00:00<?, ?it/s]

In [6]:
sys.getsizeof(fids)

15379208

In [7]:
def get_film_data(id: str):
    film_url = f'https://www.imdb.com/title/tt{id}'
    r = requests.get(
        film_url,
        headers={'User-Agent': random.choice(USER_AGENTS)}
    )
    soup = BeautifulSoup(r.text, 'html.parser').head
    ld_json_script = soup.find('script', type='application/ld+json')
    json_data = loads(ld_json_script.string)
    return json_data

In [8]:
film_id = '0910970'
walle = get_film_data(film_id)

In [9]:
walle['duration']
dt = pd.Timedelta(walle['duration'])
int(dt.total_seconds())

5880

In [14]:
test_fids = list(fids.keys())[:100]
test_fids

['0111161',
 '0109830',
 '0110912',
 '0133093',
 '0068646',
 '0120737',
 '0114369',
 '0172495',
 '0102926',
 '0120815',
 '0108052',
 '0076759',
 '0080684',
 '0071562',
 '0088763',
 '0120338',
 '0099685',
 '0110413',
 '0120382',
 '0169547',
 '0120586',
 '0103064',
 '0114814',
 '0086190',
 '0105236',
 '0112573',
 '0119217',
 '0073486',
 '0107290',
 '0082971',
 '0078748',
 '0095016',
 '0075314',
 '0088247',
 '0066921',
 '0120915',
 '0083658',
 '0097576',
 '0093058',
 '0090605',
 '0118799',
 '0144084',
 '0116282',
 '0117951',
 '0113277',
 '0054215',
 '0062622',
 '0078788',
 '0107048',
 '0073195',
 '0114746',
 '0120363',
 '0119488',
 '0119654',
 '0120735',
 '0034583',
 '0116629',
 '8111088',
 '0071853',
 '6710474',
 '0095953',
 '0057012',
 '0119116',
 '0092099',
 '0038650',
 '0101414',
 '0084787',
 '0117060',
 '0033467',
 '0120616',
 '2283362',
 '0120591',
 '0119567',
 '0091763',
 '0083866',
 '0105695',
 '0163651',
 '0032138',
 '9140554',
 '0079470',
 '0112864',
 '0096895',
 '0119094',
 '01

In [11]:
# with open('test.csv', 'w') as fw:
#     # fw.write(f'fid,name,description,ratingCount,ratingValue,contentRating,genre,keywords,duration\n')
#     for fid in test_fids:
#         json_film = get_film_data(fid)
#         fw.write(f'"{fid}","{json_film['name']}","{json_film['description']}",{json_film['aggregateRating']['ratingCount']},')
#         fw.write(f'{json_film['aggregateRating']['ratingValue']},{json_film['contentRating']},')
#         fw.write(f'"{','.join(json_film['genre'])}","{json_film['keywords']}",{int(pd.Timedelta(json_film['duration']).total_seconds())}\n')

In [16]:
def get_film_data(film_id: str):
    """Fetches film data from IMDb."""
    film_url = f'https://www.imdb.com/title/tt{film_id}'
    try:
        r = requests.get(
            film_url,
            headers={'User-Agent': random.choice(USER_AGENTS)}
        )
        if r.status_code != 200:
            print(f"Failed to fetch data for ID {film_id}. Status code: {r.status_code}")
            return None

        soup = BeautifulSoup(r.text, 'html.parser').head
        ld_json_script = soup.find('script', type='application/ld+json')
        if not ld_json_script:
            print(f"No ld+json script found for ID {film_id}")
            return None

        json_data = loads(ld_json_script.string)
        return json_data

    except requests.exceptions.RequestException as e:
        print(f"Request failed for ID {film_id}: {e}")
        return None
    except Exception as e:
        print(f"An error occurred for ID {film_id}: {e}")
        return None

def process_film(fid: str):
    """Fetches film data and formats it as a CSV row."""
    json_film = get_film_data(fid)
    if not json_film:
        return None
    
    try:
        name = unescape(json_film.get('name', ''))
        description = unescape(json_film.get('description', ''))
        rating_count = json_film.get('aggregateRating', {}).get('ratingCount', 0)
        rating_value = json_film.get('aggregateRating', {}).get('ratingValue', 0.0)
        content_rating = json_film.get('contentRating', '')
        genre = unescape(','.join(json_film.get('genre', [])))
        keywords = unescape(json_film.get('keywords', ''))
        duration = int(pd.Timedelta(json_film.get('duration', 'PT0S')).total_seconds())

        return f'{fid},"{name}","{description}",{rating_count},{rating_value},{content_rating},"{genre}","{keywords}",{duration}\n'
    except Exception as e:
        print(f"Error processing data for ID {fid}: {e}")
        return None

def main_parallel(test_fids):
    """Writes film data to a CSV file in parallel."""
    with open('test.csv', 'w') as fw:
        # Write the header
        fw.write('fid,name,description,ratingCount,ratingValue,contentRating,genre,keywords,duration\n')

        # Use a ThreadPoolExecutor for parallel processing
        with concurrent.futures.ThreadPoolExecutor() as executor:
            # Map film IDs to their processed data rows
            futures = {executor.submit(process_film, fid): fid for fid in test_fids}
            
            for future in concurrent.futures.as_completed(futures):
                result = future.result()
                if result:
                    fw.write(result)

In [17]:
main_parallel(test_fids)